Imports

In [17]:
import os
import random
import numpy as np
import pandas as pd
import plotly.graph_objects as go

from typing import List, Optional
from dataclasses import dataclass

Constants

In [18]:
PMC_CONVENTIONAL = 4362.47
PMC_OPERATIVE_69 = 2024.481
PMC_OPERATIVE_138 = 2513.486

PQ_QUANTITY = 72

Classes and functions

In [19]:
from dataclasses import dataclass
import pandas as pd

@dataclass
class BarData:
    '''
    A dataclass that retains relevant information about system's bars.

    This dataclass stores information about system bars, including the file name, name, voltage class, and a DataFrame.

    Attributes:
        _file_name (str): The file name associated with the bar data.

    Properties:
        file_name (str): The file name associated with the bar data.
        name (str): The name extracted from the file name (excluding extension).
        voltage_class (str): The voltage class extracted from the file name.
        df (pd.DataFrame): A DataFrame containing the bar data.
    '''

    _file_name: str

    @property
    def file_name(self) -> str:
        '''str: The file name associated with the bar data.'''
        return self._file_name

    @property
    def name(self) -> str:
        '''str: The name extracted from the file name (excluding extension).'''
        return self._file_name[9:-4]

    @property
    def voltage_class(self) -> str:
        '''str: The voltage class extracted from the file name.'''
        return self.name[-3:]

    @property
    def df(self) -> pd.DataFrame:
        '''pd.DataFrame: A DataFrame containing the bar data.'''
        return self.read_csv(self._file_name)

    def read_csv(self, _file_name: str) -> pd.DataFrame:
        '''
        Read bar data from a CSV file and return it as a DataFrame.

        Args:
            _file_name (str): The file name of the CSV file.

        Returns:
            pd.DataFrame: A DataFrame containing the bar data.
        '''
        df = pd.read_csv(_file_name, delimiter= ';', header= None)
        return df

    def __repr__(self) -> str:
        '''str: A string representation of the BarData instance.'''
        return f'{self.name}'

    
def get_random_sum(loads: List[int], get_amount: int) -> int:
    '''
    Calculate the sum of a random selection of load values in megawatts (MW).

    This function randomly selects a specified number of load values from the provided list and calculates their sum in megawatts (MW).

    Args:
        loads (List[int]): A list of load values in megawatts (MW).
        get_amount (int): The number of load values to randomly select and sum.

    Returns:
        int: The sum of the randomly selected load values in megawatts (MW).
    '''
    return sum(random.sample(loads, get_amount))

def increment_power(load: float,
                    increment_amount: int,
                    increment_percentage: float
                    ) -> List[float]:
    '''
    Increment a load value by a specified number and percentage multiple times.

    This function takes a starting load value and increments it by a specified number and percentage for a given number of times.

    Args:
        load (float): The initial load value.
        increment_amount (int): The number of times to increment the load.
        increment_percentage (float): The percentage by which to increment the load in each step.

    Returns:
        list: A list containing the incremented load values, including the initial load.
    '''
    incremented_values = [load]

    for _ in range(increment_amount):
        load *= (1 + increment_percentage)
        incremented_values.append(load)

    return incremented_values

def make_matrices(max_power: float,
                  load_value: List[int],
                  charging_station_power: List[float],
                  increment: bool = False
                  ) -> tuple[List[List], List[List]]:
    '''
    Generate matrices of power and margin of stability based on max power, load, charging station power, and increment settings.

    This function calculates power and margin of stability matrices for a given load value and power value reference, considering various charging station power scenarios.

    Args
        max_power (float): Reference power value.
        load_value (float): The base load value.
        charging_station_power (list): A list of charging station power values.
        increment (bool, optional): Whether to increment load with charging station power scenarios (default is False).

    Returns:
        tuple: A tuple containing two lists of lists:
            - A power matrix: List of lists representing power values for different scenarios.
            - A margin of stability matrix: List of lists representing margin of stability values for different scenarios.
    '''
    pow_matrix = []
    met_matrix = []

    for n in range(PQ_QUANTITY):
        pow_row = []
        met_row = []

        for _ in range(200):
            charging_site_power = get_random_sum(charging_station_power, get_amount= 3) / 1000
            
            power_value = load_value + (n + 1) * charging_site_power if increment else load_value

            met_value = 100.0 * ((max_power - power_value) / max_power)

            pow_row.append(power_value)
            met_row.append(met_value)
        pow_matrix.append(pow_row)
        met_matrix.append(met_row)
    
    return pow_matrix, met_matrix

def filter_by_voltage(bars: List[BarData],
                      voltage_class: Optional[str] = None
                      ) -> List[BarData]:
    '''
    Filter a list of BarData instances by their voltage_class attribute.

    Args:
        bars (List[BarData]): List of BarData instances to filter.
        voltage_class (str): The voltage class to filter by. Defaults to None.

    Returns:
        List[BarData]: List of filtered BarData instances.
    '''

    if voltage_class is None:
        return bars
    
    filtered_bars = [bar for bar in bars if bar.voltage_class == voltage_class]
    
    return filtered_bars

BarData objects

In [20]:
folder_path = r'PVs_csvs'

bars = []

for file in os.listdir(folder_path):
    if file.endswith('.csv'):
        file_path = os.path.join(folder_path, file)
        
        bar = BarData(file_path)

        bars.append(bar)

Load defintions

In [21]:
heavy_load = 1642.69
charging_station_power = [114, 147, 180, 213]

incremented_loads = increment_power(heavy_load, 5, 0.082)

incremented_loads

[1642.69,
 1777.3905800000002,
 1923.1366075600004,
 2080.8338093799207,
 2251.4621817490743,
 2436.0820806524985]

Yearly analyses

In [22]:
years = [2026, 2028, 2030, 2032, 2034, 2036]

pow_matrix_incremented = {}
met_matrix_incremented = {}
pow_matrix_unincremented = {}
met_matrix_unincremented = {}

for year in years:
    load_data = incremented_loads[years.index(year)]
    
    pow_matrix_incremented[year], met_matrix_incremented[year] = make_matrices(PMC_CONVENTIONAL,
                                                                               load_data,
                                                                               charging_station_power,
                                                                               True)
    
    pow_matrix_unincremented[year], met_matrix_unincremented[year] = make_matrices(PMC_CONVENTIONAL, 
                                                                                   load_data,
                                                                                   charging_station_power,
                                                                                   False)


In [23]:
years = [2026, 2028, 2030, 2032, 2034, 2036]

pow_matrix_incremented_69 = {}
met_matrix_incremented_69 = {}
pow_matrix_unincremented_69 = {}
met_matrix_unincremented_69 = {}

for year in years:
    load_data = incremented_loads[years.index(year)]
    
    pow_matrix_incremented_69[year], met_matrix_incremented_69[year] = make_matrices(PMC_OPERATIVE_69,
                                                                               load_data,
                                                                               charging_station_power,
                                                                               True)
    
    pow_matrix_unincremented_69[year], met_matrix_unincremented_69[year] = make_matrices(PMC_OPERATIVE_69, 
                                                                                   load_data,
                                                                                   charging_station_power,
                                                                                   False)


In [34]:
years = [2026, 2028, 2030, 2032, 2034, 2036]

pow_matrix_incremented_138 = {}
met_matrix_incremented_138 = {}
pow_matrix_unincremented_138 = {}
met_matrix_unincremented_138 = {}

for year in years:
    load_data = incremented_loads[years.index(year)]
    
    pow_matrix_incremented_138[year], met_matrix_incremented_138[year] = make_matrices(PMC_OPERATIVE_138,
                                                                               load_data,
                                                                               charging_station_power,
                                                                               True)
    
    pow_matrix_unincremented_138[year], met_matrix_unincremented_138[year] = make_matrices(PMC_OPERATIVE_138, 
                                                                                   load_data,
                                                                                   charging_station_power,
                                                                                   False)


Plots

In [24]:
fig = go.Figure()

filtered_bars = filter_by_voltage(bars)

for index, bar in enumerate(filtered_bars):
    data = bar.df.T
    
    fig.add_trace(
        go.Scatter(x= data.values[0],
                   y= data.values[1],
                   mode= 'lines',
                   name= f'Bar {index + 1}: {bar.name}')
    )

fig.update_layout(
    xaxis_title= 'Load (MW)',
    yaxis_title= 'Voltage (pu)',
    title= 'PxV curves for all voltage classes',
    title_x= 0.5
)

fig.show()


In [37]:
fig = go.Figure()

met_per_year = []
met_per_year_69 = []
met_per_year_138 = []

for year in years:
    met_per_year.append(met_matrix_unincremented[year][0][0])
    met_per_year_69.append(met_matrix_unincremented_69[year][0][0])
    met_per_year_138.append(met_matrix_unincremented_138[year][0][0])

fig.add_trace(
    go.Scatter(x= years,
               y= met_per_year,
               name= 'Conventional VSM')
)

fig.add_trace(
    go.Scatter(x= years,
               y= met_per_year_69,
               name= '69kV operative VSM')
)

fig.add_trace(
    go.Scatter(x= years,
               y= met_per_year_138,
               name= '138kV operative VSM')
)

# fig.update_yaxes(range= [40, 65])

fig.update_layout(
    xaxis_title= 'Years',
    yaxis_title= 'Voltage margin (%)',
    title= 'Voltage stability margin over the years (unincremented load)',
    title_x= 0.5
)

fig.show()

In [26]:
fig = go.Figure()

data = pd.DataFrame.from_dict(met_matrix_incremented)

for _, (key, df) in enumerate(data.items()):
    fig.add_trace(go.Box(y= df,
                         name= key))

fig.update_layout(
    title= 'Voltage margin over the years (incremented load)',
    title_x= 0.5,
    xaxis= dict(title= 'Years'),
    yaxis=dict(
        title='Voltage margin (%)',
        tickmode='linear',  # Use a linear tick mode
        tick0=0,            # Starting tick value
        dtick=5
        )
)

fig.show()


In [27]:
target_voltages = ['069', '138']

met_matrix_filtered = {}

for voltage in target_voltages:
    met_matrix_filtered[voltage] = filter_by_voltage(bars, voltage)

In [38]:
fig = go.Figure()

filtered_bars = filter_by_voltage(bars, '230')

for index, bar in enumerate(filtered_bars):
    data = bar.df.T
    
    fig.add_trace(
        go.Scatter(x= data.values[0],
                   y= data.values[1],
                   mode= 'lines',
                   name= f'Bar {index + 1}: {bar.name}')
    )

fig.update_layout(
    xaxis_title= 'Load (MW)',
    yaxis_title= 'Voltage (pu)',
    title= 'PxV curves for all voltage classes',
    title_x= 0.5
)

fig.show()

PMC_OPERATIVO_69 = 2024.4810
PMC_OPERATIVO_138 = 2513.4860
